In [4]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# prompt: import file from this path '/content/sample_data/lasso_model.pkl'

import pickle
with open('/content/sample_data/lasso_model.pkl', 'rb') as file:
  lasso_model = pickle.load(file)
# Now you can use the lasso_model object
lasso_model


Lasso(alpha=0.1)

In [8]:
# Install Gradio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [9]:
import gradio as gr
import numpy as np
import pickle

# Load the model
with open('/content/sample_data/lasso_model.pkl', 'rb') as file:
    lasso_model = pickle.load(file)

# Options
country_options = ['Argentina', 'Australia', 'Bangladesh', 'Belgium', 'Brazil', 'Canada', 'China', 'Colombia',
                   'France', 'Germany', 'India', 'Ireland', 'Italy', 'Japan', 'Kenya', 'Mexico', 'Netherlands',
                   'Nigeria', 'Pakistan', 'Poland', 'Portugal', 'Russia', 'South Africa', 'South Korea',
                   'Spain', 'Turkey', 'United Arab Emirates',
                   'United Kingdom of Great Britain and Northern Ireland', 'United States of America', 'Other']

title_options = ['Data Analyst (Business, Marketing, Financial, Quantitative, etc)',
                 'Data Scientist', 'Data Engineer', 'Machine Learning/ MLops Engineer',
                 'Manager (Program, Project, Operations, Executive-level, etc)', 'Software Engineer', 'Other']

industry_options = ['Accounting/Finance', 'Broadcasting/Communications', 'Computers/Technology', 'Energy/Mining',
                    'Government/Public Service', 'Medical/Pharmaceutical', 'Online Service/Internet-based Services',
                    'Retail/Sales', 'Other']

ml_experience_options = [
    'No (we do not use ML methods)',
    'We use ML methods for generating insights (but do not put working models into production)',
    'We recently started using ML methods (i.e., models in production for less than 2 years)',
    'We are exploring ML methods (and may one day put a model into production)',
    'We have well established ML methods (i.e., models in production for more than 2 years)'
]

spending_options = ['$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)']

# Expected feature order
final_feature_list = ['For how many years have you used machine learning methods?',
                      'What is the size of the company where you are employed?'] + \
                     [f"In which country do you currently reside?_{c}" for c in country_options] + \
                     [f"Select the title most similar to your current role (or most recent title if retired): - Selected Choice_{t}" for t in title_options] + \
                     [f"In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice_{i}" for i in industry_options] + \
                     [f"Does your current employer incorporate machine learning methods into their business?_{m}" for m in ml_experience_options] + \
                     [f"Approximately how much money have you spent on machine learning and/or cloud computing services at home or at work in the past 5 years (approximate $USD)?\n (approximate $USD)?_{s}" for s in spending_options]

# Prediction function
def predict_salary(years_exp, company_size, country, title, industry, ml_level, spending_bracket):
    try:
        features = np.zeros(lasso_model.n_features_in_)

        def set_one_hot(template, value):
            full_feature = template.format(value)
            if full_feature in final_feature_list:
                features[final_feature_list.index(full_feature)] = 1
            else:
                print("Missing feature:", full_feature)

        features[0] = years_exp
        features[1] = company_size
        set_one_hot("In which country do you currently reside?_{}", country)
        set_one_hot("Select the title most similar to your current role (or most recent title if retired): - Selected Choice_{}", title)
        set_one_hot("In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice_{}", industry)
        set_one_hot("Does your current employer incorporate machine learning methods into their business?_{}", ml_level)
        set_one_hot("Approximately how much money have you spent on machine learning and/or cloud computing services at home or at work in the past 5 years (approximate $USD)?\n (approximate $USD)?_{}", spending_bracket)

        prediction = lasso_model.predict([features])[0]
        return f"${max(prediction, 0):,.2f}"
    except Exception as e:
        return f"Error: {e}"

# Build app layout with Blocks for vertical stacking
with gr.Blocks(title="Data Scientist Salary Projector") as demo:
    gr.Markdown("# 💼 Data Scientist Salary Projector")
    gr.Markdown("Estimate your salary using experience, job title, and industry context.")

    with gr.Column():
        years_exp = gr.Slider(0, 50, step=1, label="Years of ML Experience")
        company_size = gr.Number(label="Company Size (Employees)")
        country = gr.Dropdown(choices=country_options, label="Country of Residence")
        title = gr.Dropdown(choices=title_options, label="Current Job Title")
        industry = gr.Dropdown(choices=industry_options, label="Employer Industry")
        ml_level = gr.Dropdown(choices=ml_experience_options, label="ML Production Experience")
        spending = gr.Dropdown(choices=spending_options, label="ML/Cloud Spending (5 Years)")

        output = gr.Textbox(label="Predicted Salary")

        submit = gr.Button("Calculate Salary")

        submit.click(
            predict_salary,
            inputs=[years_exp, company_size, country, title, industry, ml_level, spending],
            outputs=output
        )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b37c9460bb393e49c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
